In [1]:
!pip3 install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.6 MB/s eta 0:00:00


In [53]:
import pandas as pd
import warnings
import re
import numpy as np
import matplotlib.pyplot as plt
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore')

from google.colab import drive
from spellchecker import SpellChecker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
domain_dataset = pd.read_csv('/content/drive/MyDrive/coursework_data_m.csv')

In [5]:
domain_dataset

,Domain,category
0,noriomatsumoto.com,1
1,haplessmales.com,1
2,shivamchatuvedi.com,1
3,dailyfreebieoffer.com,1
4,poniemall.com,1
...,...,...
10000,citymark.net,2
10001,cevon.net,2
10002,global-1.net,2
10003,mybridge.net,4


In [6]:
domain_dataset['Length'] = [0] * 10005

In [7]:
for i in range(10005):
  domain_dataset['Length'][i] = len(domain_dataset['Domain'][i].split('.')[0])

In [8]:
domain_dataset

,Domain,category,Length
0,noriomatsumoto.com,1,14
1,haplessmales.com,1,12
2,shivamchatuvedi.com,1,15
3,dailyfreebieoffer.com,1,17
4,poniemall.com,1,9
...,...,...,...
10000,citymark.net,2,8
10001,cevon.net,2,5
10002,global-1.net,2,8
10003,mybridge.net,4,8


In [9]:
domain_dataset['ExtensionScore'] = [0] * 10005

In [10]:
for i in range(10005):
  extension = domain_dataset['Domain'][i].split('.')[-1].lower()
  extension_score = 0
  if extension == 'com':
     extension_score = 3
  elif extension == 'net':
    extension_score = 2
  elif extension == 'org':
    extension_score = 1
  else:
    extension_score = 0
  domain_dataset['ExtensionScore'][i] = extension_score

In [11]:
domain_dataset

,Domain,category,Length,ExtensionScore
0,noriomatsumoto.com,1,14,3
1,haplessmales.com,1,12,3
2,shivamchatuvedi.com,1,15,3
3,dailyfreebieoffer.com,1,17,3
4,poniemall.com,1,9,3
...,...,...,...,...
10000,citymark.net,2,8,2
10001,cevon.net,2,5,2
10002,global-1.net,2,8,2
10003,mybridge.net,4,8,2


In [79]:
class TNode:
    def __init__(self):
        self.child={}
        self.isWordFinished=False
class Trie:
    def __init__(self):
        self.root=TNode()
        

    def insert(self, word: str) -> None:
        word = word.upper()
        word = word[::-1]
        cur=self.root
        for i in word:
            if i not in cur.child:
                cur.child[i]=TNode()
            cur=cur.child[i]
        cur.isWordFinished=True

    def search(self, word: str) -> bool:
        word = word.upper()
        word = word[::-1]
        cur=self.root
        for i in word:
            if i not in cur.child:
                return False
            cur=cur.child[i]
        return cur.isWordFinished     

    def getBrokenWords(self, word):
      word = word.upper()
      word_len = len(word)
      if word_len < 3:
        return [word]
      dp = [[False, -1]] * word_len
      for i in range(1, word_len):
        if word[i] not in self.root.child:
          continue
        cur = self.root.child[word[i]]
        for j in range(i-1, -1, -1):
          if word[j] not in cur.child:
            break
          cur = cur.child[word[j]]
          if cur.isWordFinished and j==0:
            dp[i] = [True, 0]
            break
          if cur.isWordFinished and (j > 0) and dp[j-1][0]:
            dp[i] = [True, j]
      if not dp[word_len-1][0]:
        return [word]
      broken_words = []
      i = word_len
      j = dp[i-1][1]
      while i>0:
        broken_words.append(word[j:i])
        i = j
        if j <=0:
          break
        j = dp[i-1][1]
      return broken_words[::-1]

In [33]:
dictionary_words = open('/content/drive/MyDrive/final_dictionary.txt', 'r').read().split('\n')

In [34]:
len(dictionary_words)

373893

In [15]:
print(dictionary_words[:10])

['AAA', 'AAH', 'AAHED', 'AAHING', 'AAHS', 'AAL', 'AALII', 'AALIIS', 'AALS', 'AAM']


In [80]:
trie = Trie()
for word in dictionary_words:
  trie.insert(word)

In [128]:
trie.search('COLLEGE')

True

In [81]:
print(trie.getBrokenWords('busytoday'))
print(trie.getBrokenWords('forhonor'))
print(trie.getBrokenWords('goodfriday'))
print(trie.getBrokenWords('badsundayyesterday'))
print(trie.getBrokenWords('amongus'))
print(trie.getBrokenWords('shoppingmall'))
print(trie.getBrokenWords('upgradetolinux'))
print(trie.getBrokenWords('todayissaturday'))
print(trie.getBrokenWords('citymall'))

['BUSY', 'TODAY']
['FOR', 'HONOR']
['GOOD', 'FRIDAY']
['BAD', 'SUNDAY', 'YESTERDAY']
['AM', 'ON', 'GUS']
['SHOPPING', 'MALL']
['UPGRADE', 'TO', 'LINUX']
['TODA', 'YIS', 'SATURDAY']
['CITY', 'MALL']


In [46]:
domain_dataset

,Domain,category,Length,ExtensionScore
0,noriomatsumoto.com,1,14,3
1,haplessmales.com,1,12,3
2,shivamchatuvedi.com,1,15,3
3,dailyfreebieoffer.com,1,17,3
4,poniemall.com,1,9,3
...,...,...,...,...
10000,citymark.net,2,8,2
10001,cevon.net,2,5,2
10002,global-1.net,2,8,2
10003,mybridge.net,4,8,2


In [48]:
domain_dataset['SpecialCharactersCount'] = [0] * 10005

In [49]:
for i in range(10005):
  count = 0
  domain_name = str(domain_dataset['Domain'][i].split('.')[0])
  for ch in domain_name:
    if not ch.isalpha(): #If the character is not an alphabet then increasing count of special characters
      count += 1
  domain_dataset['SpecialCharactersCount'][i] = count

In [50]:
domain_dataset

,Domain,category,Length,ExtensionScore,SpecialCharactersCount
0,noriomatsumoto.com,1,14,3,0
1,haplessmales.com,1,12,3,0
2,shivamchatuvedi.com,1,15,3,0
3,dailyfreebieoffer.com,1,17,3,0
4,poniemall.com,1,9,3,0
...,...,...,...,...,...
10000,citymark.net,2,8,2,0
10001,cevon.net,2,5,2,0
10002,global-1.net,2,8,2,2
10003,mybridge.net,4,8,2,0


In [135]:
domain_dataset['DictionaryWordsBroken'] = [[]] * 10005

In [138]:
domain_dataset

,Domain,category,Length,ExtensionScore,SpecialCharactersCount,DictionaryWordsBroken
0,apple-is-ghghghg.com,1,14,3,0,[]
1,haplessmales.com,1,12,3,0,[]
2,shivamchatuvedi.com,1,15,3,0,[]
3,dailyfreebieoffer.com,1,17,3,0,[]
4,poniemall.com,1,9,3,0,[]
...,...,...,...,...,...,...
10000,citymark.net,2,8,2,0,[]
10001,cevon.net,2,5,2,0,[]
10002,global-1.net,2,8,2,2,[]
10003,mybridge.net,4,8,2,0,[]


In [67]:
def split_string_by_special_characters(word):
  word = re.sub('[^a-zA-Z \n\.]', '-', word)
  res = [x for x in word.split('-') if len(x) > 0]
  return res

In [88]:
def get_corrected_words(word_list, spell):
  res = []
  for word in word_list:
    corrected_word = spell.correction(word)
    if corrected_word is None:
      corrected_word = ''
    res.append(corrected_word)
  return res

In [164]:
def get_alpha_word(word):
  res = ''
  for i in word:
    if i.isalpha():
      res += i
  return res.upper()

In [192]:
def count_commas(word):
  count = 0
  for i in word:
    if i == ',':
      count += 1
  return count

In [193]:
def count_alphas(word):
  count = 0
  for i in word:
    if i.isalpha():
      count += 1
  return count

In [307]:
domain_dataset = pd.read_csv('/content/drive/MyDrive/processed_dataset.csv')

In [308]:
domain_dataset

,Domain,category,Length,ExtensionScore,SpecialCharactersCount,DictionaryWordsBroken,EditDistance,AvgMonthlySearchVolume
0,noriomatsumoto.com,1,14,3,0,"['NO', 'RIO', 'MAT', 'SUM', 'OTO']",0,0
1,haplessmales.com,1,12,3,0,"['HAPLESS', 'MALES']",0,0
2,shivamchatuvedi.com,1,15,3,0,"['SHIV', 'AM', 'CHA', 'TU', 'VE', 'DI']",0,0
3,dailyfreebieoffer.com,1,17,3,0,"['DAILY', 'FREEBIE', 'OFFER']",0,0
4,poniemall.com,1,9,3,0,"['PON', 'IE', 'MALL']",0,0
...,...,...,...,...,...,...,...,...
10000,citymark.net,2,8,2,0,"['CITY', 'MARK']",0,5000
10001,cevon.net,2,5,2,0,"['CE', 'VON']",0,500
10002,global-1.net,2,8,2,2,['GLOBAL'],0,5000
10003,mybridge.net,4,8,2,0,"['MY', 'BRIDGE']",0,50000


In [155]:
# Function to calculate edit distance between two words
def get_edit_distance(word1, word2):
    if word1 == word2:
        return 0

    word_len1 = len(word1)
    word_len2 = len(word2)

    dp = [[0 for i in range(word_len1 + 1)]
          for j in range(2)];

    for i in range(0, word_len1 + 1):
        dp[0][i] = i

    for i in range(1, word_len2 + 1):
        for j in range(0, word_len1 + 1):
            if (j == 0):
                dp[i % 2][j] = i
            elif word1[j - 1] == word2[i - 1]:
                dp[i % 2][j] = dp[(i - 1) % 2][j - 1]
            else:
                dp[i % 2][j] = (1 + min(dp[(i - 1) % 2][j],
                                        min(dp[i % 2][j - 1],
                                            dp[(i - 1) % 2][j - 1])))

    return dp[word_len2 % 2][word_len1]

In [162]:
for i in range(10005):
  domain_dataset['DictionaryWordsBroken'][i] = domain_dataset['DictionaryWordsBroken'][i].upper()

In [167]:
domain_dataset['EditDistance'] = [0] * 10005

In [170]:
for i in range(10005):
  domain_name = str(domain_dataset['Domain'][i].split('.')[0])
  domain_name = get_alpha_word(domain_name)
  corrected_word = get_alpha_word(domain_dataset['DictionaryWordsBroken'][i])
  domain_dataset['EditDistance'][i] = get_edit_distance(domain_name, corrected_word)

In [171]:
domain_dataset

,Domain,category,Length,ExtensionScore,SpecialCharactersCount,DictionaryWordsBroken,EditDistance
0,noriomatsumoto.com,1,14,3,0,"['NO', 'RIO', 'MAT', 'SUM', 'OTO']",0
1,haplessmales.com,1,12,3,0,"['HAPLESS', 'MALES']",0
2,shivamchatuvedi.com,1,15,3,0,"['SHIV', 'AM', 'CHA', 'TU', 'VE', 'DI']",0
3,dailyfreebieoffer.com,1,17,3,0,"['DAILY', 'FREEBIE', 'OFFER']",0
4,poniemall.com,1,9,3,0,"['PON', 'IE', 'MALL']",0
...,...,...,...,...,...,...,...
10000,citymark.net,2,8,2,0,"['CITY', 'MARK']",0
10001,cevon.net,2,5,2,0,"['CE', 'VON']",0
10002,global-1.net,2,8,2,2,['GLOBAL'],0
10003,mybridge.net,4,8,2,0,"['MY', 'BRIDGE']",0


In [309]:
domain_dataset['IndividualWordsCount'] = 0 * 10005

In [310]:
domain_dataset

,Domain,category,Length,ExtensionScore,SpecialCharactersCount,DictionaryWordsBroken,EditDistance,AvgMonthlySearchVolume,IndividualWordsCount
0,noriomatsumoto.com,1,14,3,0,"['NO', 'RIO', 'MAT', 'SUM', 'OTO']",0,0,0
1,haplessmales.com,1,12,3,0,"['HAPLESS', 'MALES']",0,0,0
2,shivamchatuvedi.com,1,15,3,0,"['SHIV', 'AM', 'CHA', 'TU', 'VE', 'DI']",0,0,0
3,dailyfreebieoffer.com,1,17,3,0,"['DAILY', 'FREEBIE', 'OFFER']",0,0,0
4,poniemall.com,1,9,3,0,"['PON', 'IE', 'MALL']",0,0,0
...,...,...,...,...,...,...,...,...,...
10000,citymark.net,2,8,2,0,"['CITY', 'MARK']",0,5000,0
10001,cevon.net,2,5,2,0,"['CE', 'VON']",0,500,0
10002,global-1.net,2,8,2,2,['GLOBAL'],0,5000,0
10003,mybridge.net,4,8,2,0,"['MY', 'BRIDGE']",0,50000,0


In [311]:
for i in range(10005):
  comma_count = count_commas(domain_dataset['DictionaryWordsBroken'][i])
  domain_dataset['IndividualWordsCount'][i] =  comma_count + 1

In [283]:
domain_dataset

,Domain,category,Length,ExtensionScore,SpecialCharactersCount,DictionaryWordsBroken,EditDistance,AvgMonthlySearchVolume,IndividualWordsCount
0,noriomatsumoto.com,1,14,3,0,"['NO', 'RIO', 'MAT', 'SUM', 'OTO']",0,0,5
1,haplessmales.com,1,12,3,0,"['HAPLESS', 'MALES']",0,0,2
2,shivamchatuvedi.com,1,15,3,0,"['SHIV', 'AM', 'CHA', 'TU', 'VE', 'DI']",0,0,6
3,dailyfreebieoffer.com,1,17,3,0,"['DAILY', 'FREEBIE', 'OFFER']",0,0,3
4,poniemall.com,1,9,3,0,"['PON', 'IE', 'MALL']",0,0,3
...,...,...,...,...,...,...,...,...,...
10000,citymark.net,2,8,2,0,"['CITY', 'MARK']",0,5000,2
10001,cevon.net,2,5,2,0,"['CE', 'VON']",0,500,2
10002,global-1.net,2,8,2,2,['GLOBAL'],0,5000,1
10003,mybridge.net,4,8,2,0,"['MY', 'BRIDGE']",0,50000,2


In [284]:
domain_dataset.isnull().sum()

Domain                    0
category                  0
Length                    0
ExtensionScore            0
SpecialCharactersCount    0
DictionaryWordsBroken     0
EditDistance              0
AvgMonthlySearchVolume    0
IndividualWordsCount      0
dtype: int64

In [ ]:
domain_dataset.to_csv('processed_dataset.csv', encoding='utf-8', index=False)

In [300]:
# Selecting input features
features = ['Length', 'EditDistance',
            'AvgMonthlySearchVolume', 'ExtensionScore',
            'SpecialCharactersCount', 'IndividualWordsCount']

In [301]:
# Target Value
target_value = ['category']

In [302]:
X = domain_dataset[features]
X.head()

,Length,EditDistance,AvgMonthlySearchVolume,ExtensionScore,SpecialCharactersCount,IndividualWordsCount
0,14,0,0,3,0,5
1,12,0,0,3,0,2
2,15,0,0,3,0,6
3,17,0,0,3,0,3
4,9,0,0,3,0,3


In [303]:
Y = domain_dataset[target_value]
Y.head()

,category
0,1
1,1
2,1
3,1
4,1


In [304]:
dtc = DecisionTreeClassifier() 
dtc.fit(X, Y)
dtc.score(X, Y) * 100

72.64367816091955

In [305]:
rfc = RandomForestClassifier()
rfc.fit(X, Y)
rfc.score(X, Y) * 100

72.64367816091955

In [306]:
knn = KNeighborsClassifier() 
knn.fit(X, Y)
knn.score(X, Y) * 100

67.47626186906547